In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import os

In [3]:
input_path = '/content/drive/MyDrive/機械学習/Competitions/Signate/医学論文の自動仕分けチャレンジ/input'

fasttext_df = pd.read_feather(os.path.join(input_path,'abstract_fasttxt_df.feather'))
word2vec_df = pd.read_feather(os.path.join(input_path,'abstract_df.feather'))
kmeans_df = pd.read_feather(os.path.join(input_path,'kmeans_tfidf_abstract_df.feather'))
train_df = pd.read_csv(os.path.join(input_path,'train.csv'))

In [4]:
print(train_df.shape)
print(fasttext_df.shape)
print(word2vec_df.shape)
print(kmeans_df.shape)

(27145, 4)
(67979, 200)
(67979, 200)
(67979, 21)


In [8]:
kmeans_df.head()

,kmeans_tfidf_abstract_0,kmeans_tfidf_abstract_1,kmeans_tfidf_abstract_2,kmeans_tfidf_abstract_3,kmeans_tfidf_abstract_4,kmeans_tfidf_abstract_5,kmeans_tfidf_abstract_6,kmeans_tfidf_abstract_7,kmeans_tfidf_abstract_8,kmeans_tfidf_abstract_9,kmeans_tfidf_abstract_10,kmeans_tfidf_abstract_11,kmeans_tfidf_abstract_12,kmeans_tfidf_abstract_13,kmeans_tfidf_abstract_14,kmeans_tfidf_abstract_15,kmeans_tfidf_abstract_16,kmeans_tfidf_abstract_17,kmeans_tfidf_abstract_18,kmeans_tfidf_abstract_19
0,1.386608,1.003365,1.389208,0.992143,1.054013,1.016412,1.048969,1.396283,1.115732,1.095984,1.017924,1.037273,1.010913,0.960975,1.059318,1.056373,1.158939,1.051931,1.005981,1.112097
1,1.365999,1.006034,1.404026,0.994194,1.083570,1.009034,1.042399,1.398144,1.099274,1.094133,1.016916,1.033461,1.027384,0.967347,0.864663,1.037493,1.156722,1.038672,0.984402,1.129536
2,1.411953,1.005403,1.370214,0.987388,1.064798,0.995703,1.026050,1.359641,1.120877,1.068689,0.998698,1.029837,0.977948,0.991881,1.066061,1.044355,1.140512,1.061001,1.013964,1.131192
3,1.391809,1.005049,1.371373,0.987164,1.077094,1.002286,1.036391,1.380212,1.112608,1.073215,0.941248,1.021423,1.002705,1.005494,1.062657,1.049682,1.131384,1.052393,1.005406,1.140270
4,1.000000,0.219747,1.000000,0.068370,0.457727,0.277664,0.353770,0.983037,0.529567,0.477838,0.259298,0.323757,0.293467,0.247502,0.414487,0.404264,0.609567,0.394354,0.243737,0.591165


In [7]:
delimit_num = train_df.shape[0]
train_fasttext_abstract = fasttext_df.iloc[:delimit_num,:]
train_word2vec_abstract = word2vec_df.iloc[:delimit_num,:]
test_fasttext_abstract = fasttext_df.iloc[delimit_num:,:]
test_word2vec_abstract = word2vec_df.iloc[delimit_num:,:]
kmeans_df.drop('id',axis=1,inplace=True)
train_kmeans = kmeans_df.iloc[:delimit_num,:]
test_kmeans = kmeans_df.iloc[delimit_num:,:]

In [ ]:
x_bow_train,x_bow_valid,y_bow_train,y_bow_valid = train_test_split(train_fasttext_abstract,train_df['judgement'],test_size=0.3,random_state=0)

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=6,n_estimators=1000,n_jobs=-1)
model.fit(x_bow_train,y_bow_train)
pred = model.predict(x_bow_valid)

In [ ]:
from sklearn.metrics import fbeta_score

score = fbeta_score(y_bow_valid,pred,beta=7.0)

In [ ]:
print(score)

0.3536147283340817


In [ ]:
x_bow_train,x_bow_valid,y_bow_train,y_bow_valid = train_test_split(train_word2vec_abstract,train_df['judgement'],test_size=0.3,random_state=0)

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=6,n_estimators=1000,n_jobs=-1)
model.fit(x_bow_train,y_bow_train)
pred = model.predict(x_bow_valid)

In [ ]:
score = fbeta_score(y_bow_valid,pred,beta=7.0)
print("word2vec model: ",score)

word2vec model:  0.3759820426487093


In [ ]:
model.fit(train_word2vec_abstract,train_df['judgement'])
prediction_sub = model.predict(test_word2vec_abstract)

In [ ]:
test_df = pd.read_csv(os.path.join(input_path,'test.csv'))

sample_submit_df = pd.DataFrame([test_df['id'], prediction_sub]).T
sample_submit_df.to_csv('/content/drive/MyDrive/機械学習/Competitions/Signate/医学論文の自動仕分けチャレンジ/output/sample_submit(word2vec_abstract).csv', header=None, index=None)

In [ ]:
sample_submit_df.head()

,id,Unnamed 0
0,27145,0
1,27146,0
2,27147,0
3,27148,0
4,27149,0


In [9]:
x_bow_train,x_bow_valid,y_bow_train,y_bow_valid = train_test_split(train_kmeans,train_df['judgement'],test_size=0.3,random_state=0)

In [10]:
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=6,n_estimators=1000,n_jobs=-1)
model.fit(x_bow_train,y_bow_train)
pred = model.predict(x_bow_valid)

In [12]:
from sklearn.metrics import fbeta_score

score = fbeta_score(y_bow_valid,pred,beta=7.0)
print("kmeans model: ",score)

kmeans model:  0.14669374858948317
